<a href="https://colab.research.google.com/github/GreihMurray/NLP-4/blob/master/anything_goes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.models import Sequential
import keras
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tqdm import tqdm
from math import log2
import json

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


Heavily based on https://www.analyticsvidhya.com/blog/2019/08/comprehensive-guide-language-model-nlp-python-code/#h2_7

In [118]:
def encode(grams, raw_data, loader=False):
    chars = sorted(list(set(raw_data)))
    mapping = dict((c, i) for i, c in enumerate(chars))

    if loader == True:
        with open('/content/gdrive/MyDrive/Colab_Notebooks/NLP/swahili/swmap.json') as infile:
          mapping = json.load(infile) 

    sequences = list()
    for line in tqdm(grams, desc='Encoding'):
        # integer encode line
        encoded_seq = [mapping[char] for char in line]
        # store
        sequences.append(encoded_seq)
    return sequences, mapping

Based on code from https://www.analyticsvidhya.com/blog/2019/08/comprehensive-guide-language-model-nlp-python-code/#h2_7

In [4]:
def build_model(vocab):
    model = Sequential()
    model.add(Embedding(vocab, 20, input_length=GRAMS-1, trainable=True))
    model.add(GRU(25, recurrent_dropout=0.1, dropout=0.1))
    model.add(Dense(vocab, activation='softmax'))

    model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')

    return model

original

In [5]:
def read_file(file_name):
    f = open(file_name, "r")

    full_text = f.read().split(" ")
    split_data = []

    for line in full_text: #tqdm(full_text, desc='Splitting words'):
        split_data.append(line.lower().strip())

    train = ' '.join(split_data[:int(len(split_data) * 0.3)])
    test = ' '.join(split_data[int(len(split_data) * 0.5):])

    return train, test

original

In [6]:
def read_test_data(file_name):
    f = open(file_name, "r")

    full_text = f.read().split(" ")
    split_data = []

    for line in full_text: #tqdm(full_text, desc='Splitting words'):
        split_data.append(line.lower().strip())

    return ' '.join(split_data)

original

In [7]:
def gen_n_grams(data, n=3):
    descript = "Generating " + str(n) + " Grams:"

    n_grams = [''.join(data[i:i+n]) for i in tqdm(range(len(data) - n + 1), desc=descript)]

    return n_grams

In [92]:
count = {}

def main():
    data, hold_out = read_file('/content/gdrive/MyDrive/Colab_Notebooks/NLP/swahili/cwe-train.txt')

    n_grams = gen_n_grams(data, GRAMS)

    for gram in tqdm(n_grams):
      if gram in count.keys():
        continue
      count[gram] = n_grams.count(gram)

    n_grams, mapping = encode(n_grams, data)

    with open('/content/gdrive/MyDrive/Colab_Notebooks/NLP/swahili/cwmap.json', "w") as outfile:
      json.dump(mapping, outfile)

  # Below code from https://www.analyticsvidhya.com/blog/2019/08/comprehensive-guide-language-model-nlp-python-code/#h2_7
    vocab = len(mapping)
    sequences = np.array(n_grams)
    # create X and y
    x, y = sequences[:, :-1], sequences[:, -1]
    # one hot encode y
    y = to_categorical(y, num_classes=vocab)
    # create train and validation sets
    x_tr, x_val, y_tr, y_val = train_test_split(x, y, test_size=0.1, random_state=42)

    print('Train shape:', x_tr.shape, 'Val shape:', x_val.shape)

    model = build_model(vocab)

  # Original addition
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

    model.fit(x_tr, y_tr, epochs=4, verbose=1, validation_data=(x_val, y_val), callbacks=stop_early, batch_size=125)
    model.save('/content/gdrive/MyDrive/Colab_Notebooks/NLP/swahili/TESTMODEL')


In [119]:
GRAMS = 6

In [104]:
main()

  6%|▌         | 10192/181250 [00:46<12:56, 220.43it/s]


KeyboardInterrupt: ignored

Original

In [62]:
def calc_entropy(preds, mapping, sequences, vocab):
    entropy = 0

    count = 0
    pred_len = len(preds)

    keys = sequences[:, -1]

    for i in range(0, len(sequences)):
      entropy -= (1/(pred_len)) * log2(preds[i][keys[i]])

    return entropy

Original

In [120]:
def load_my_model():
    model = keras.models.load_model('/content/gdrive/MyDrive/Colab_Notebooks/NLP/swahili/sw_act_model500')

    data = read_test_data('/content/gdrive/MyDrive/Colab_Notebooks/NLP/swahili/sw-test.txt')

    n_grams = gen_n_grams(data, GRAMS)
    clean = n_grams
    n_grams, mapping = encode(n_grams, data, loader=True)

    vocab = len(mapping)
    sequences = np.array(n_grams)

    seqs = sequences[:,:-1]

    preds = model.predict(seqs)

    entropy = calc_entropy(preds, mapping, sequences, vocab)
    print('\n', entropy)
    

In [121]:
load_my_model()

Encoding: 100%|██████████| 3451377/3451377 [00:09<00:00, 371260.81it/s]


107856/107856 [==============================] - 284s 3ms/step

 2.789063893279167


In [102]:
print(count['san'])

235


In [ ]:
# List of models & Performance (KWERE) (12 grams)
  # act_model - 1.235 entropy (Batch size 250)
  # act_model500 - 1.282 entropy (Batch size 500)
  # act_model125 - 1.233 entropy (Batch size 125)
  # act_model50 - 1.201 entropy (Batch size 50)

In [ ]:
# List of models & Performance (SWAHILI) (6 grams)
  # act_model - 
  # sw_act_model500 - 1.474 entropy (Batch size 500)
  # act_model125 - 
  # act_model50 - 